**Multimodel conversation**

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
import time

In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")

In [3]:
class DebateOrchestrator:
    def __init__(self):
        # Initialize clients
        self.openai_client = OpenAI()
        self.claude_client = anthropic.Anthropic()

    def generate_system_prompts(self, topic, word_count, gpt_side, claude_side, gpt_personality, claude_personality):
        gpt_system_prompt = (
            f"You are in a debate about {topic}. "
            f"Respond with EXACTLY {word_count} words. "
            f"You are the {gpt_side} side. "
            f"Be concise, clear, and direct in your {word_count}-word counterargument. "
            f"Your personality should be {gpt_personality}"
        )
        
        claude_system_prompt = (
            f"You are in a debate about {topic}. "
            f"Respond with EXACTLY {word_count} words. "
            f"You are the {claude_side} side. "
            f"Be concise, clear, and direct in your {word_count}-word argument. "
            f"Your personality should be {claude_personality}"
        )
        
        return gpt_system_prompt, claude_system_prompt

    def call_gpt(self, claude_messages, gpt_system_prompt):
        messages = [
            {"role": "system", "content": gpt_system_prompt}
        ]
        
        for claude_msg in claude_messages:
            messages.append({"role": "user", "content": claude_msg})
        
        completion = self.openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=100,
            temperature=0.7
        )
        
        return completion.choices[0].message.content

    def call_claude(self, gpt_messages, claude_system_prompt):
        messages = [
            {"role": "user", "content": "The topic is discussed"}
        ]
        
        for gpt_msg in gpt_messages:
            messages.append({"role": "user", "content": gpt_msg})
        
        message = self.claude_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=100,
            temperature=1,
            system=claude_system_prompt,
            messages=messages
        )
        
        return message.content[0].text

    def conduct_streaming_debate(self, topic, word_count, gpt_side, claude_side, gpt_personality, claude_personality, num_rounds=5):
        # Generate system prompts based on inputs
        gpt_system_prompt, claude_system_prompt = self.generate_system_prompts(
            topic, word_count, gpt_side, claude_side, gpt_personality, claude_personality
        )
        
        gpt_messages = []
        claude_messages = []
        
        # Yield initial setup message
        yield f"🏁 Debate Topic: {topic}\n"
        yield f"📏 Word Count: {word_count}\n"
        yield f"🤖 Claude Side: {claude_side}\n"
        yield f"💻 GPT Side: {gpt_side}\n\n"
        
        for round in range(num_rounds):
            # Claude always goes first
            yield f"{'='*20} Round {round+1} {'='*20}\n"
            
            # Claude's turn
            claude_response = self.call_claude(gpt_messages, claude_system_prompt)
            claude_messages.append(claude_response)
            
            # Stream Claude's response
            yield f"Claude ({claude_side}, Round {round+1}): "
            for char in claude_response:
                yield char
                time.sleep(0.02)
            yield "\n\n"
            
            # GPT's turn
            yield f"GPT ({gpt_side}, Round {round+1}): "
            gpt_response = self.call_gpt(claude_messages, gpt_system_prompt)
            gpt_messages.append(gpt_response)
            
            # Stream GPT's response
            for char in gpt_response:
                yield char
                time.sleep(0.02)
            yield "\n\n"


In [4]:
def create_gradio_interface():
    orchestrator = DebateOrchestrator()
    
    def run_debate(topic, word_count, gpt_side, claude_side, gpt_personality, claude_personality, rounds):
        # Wrap the generator in a list to ensure compatibility
        return ''.join(
            orchestrator.conduct_streaming_debate(
                topic, word_count, gpt_side, claude_side, 
                gpt_personality, claude_personality, 
                num_rounds=rounds
            )
        )
    
    # Create Gradio interface
    with gr.Blocks() as demo:
        gr.Markdown("# Chatbot Debate Simulator")
        
        with gr.Row():
            # Topic input (text box without predefined value)
            topic_input = gr.Textbox(label="Debate Topic", interactive=True)
            
            # Word count input (number input)
            word_count_input = gr.Number(label="Word Count per Response", interactive=True)
        
        with gr.Row():
            # GPT Side dropdown
            gpt_side_input = gr.Dropdown(
                label="GPT Side", 
                choices=["Affirmative", "Opposition"], 
                interactive=True
            )
            
            # Claude Side dropdown
            claude_side_input = gr.Dropdown(
                label="Claude Side", 
                choices=["Affirmative", "Opposition"], 
                interactive=True
            )
        
        with gr.Row():
            # GPT Personality (text box without predefined value)
            gpt_personality_input = gr.Textbox(label="GPT Personality", interactive=True)
            
            # Claude Personality (text box without predefined value)
            claude_personality_input = gr.Textbox(label="Claude Personality", interactive=True)
        
        with gr.Row():
            # Number input for rounds
            rounds_input = gr.Number(label="Number of Rounds", interactive=True)
        
        # Debate output with streaming
        debate_output = gr.Textbox(label="Debate Transcript", interactive=False)
        
        # Run debate button
        run_button = gr.Button("Start Debate")
        run_button.click(
            fn=run_debate, 
            inputs=[
                topic_input, word_count_input, 
                gpt_side_input, claude_side_input, 
                gpt_personality_input, claude_personality_input, 
                rounds_input
            ], 
            outputs=debate_output
        )
    
    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
